In [1]:
import os

from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "TTS_training"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [2]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech",
    meta_file_train="metadata.csv",
    path="data",
)

In [4]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=400,
    text_cleaner=None,
    use_phonemes=False,
    phoneme_language=None,
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

In [5]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [6]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [7]:
# Full dataset
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 9445 files in C:\Users\david\dev\faks\coquis-tts\data


In [8]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [9]:
# Enable cuDNN benchmark for better performance with varying input sizes
import torch
torch.backends.cudnn.benchmark = True
torch.cuda.empty_cache()
torch.cuda.synchronize()

In [10]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(
        continue_path="TTS_training/run-May-05-2025_06+15PM-0000000"
    ),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 16
 | > Num. of Torch Threads: 8
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: True
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=TTS_training/run-May-05-2025_06+15PM-0000000
C:\Users\david\dev\faks\coquis-tts\.venv\Lib\site-packages\trainer\trainer.py:552: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
 > Restoring from checkpoint_23913.pth ...
 > Restoring Model...
 > Restoring Optimizer...
 > Restoring Scaler...
 > Model restored from step 23913
C:\Users\david\dev\faks\coquis-tts\.venv\Lib\site-packages\trainer\trainer.py:561: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.Gr

In [11]:
# Inspect data
from TTS.tts.datasets import load_tts_samples

samples = load_tts_samples(config.datasets[0], eval_split=False)

 | > Found 9445 files in C:\Users\david\dev\faks\coquis-tts\data


In [12]:
trainer.fit()

 > Restoring best loss from best_model_23733.pth ...
 > Starting with loaded last best loss {'train_loss': 0.2604999998428301, 'eval_loss': None}

 > EPOCH: 0/200
 --> TTS_training/run-May-05-2025_06+15PM-0000000




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: False
| > Number of instances : 9351



 > TRAINING (2025-05-05 23:18:43) 


 | > Preprocessing samples
 | > Max text length: 207
 | > Min text length: 5
 | > Avg text length: 75.02267137204576
 | 
 | > Max audio length: 242572.0
 | > Min audio length: 22293.0
 | > Avg audio length: 125735.99369051438
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


C:\Users\david\dev\faks\coquis-tts\.venv\Lib\site-packages\TTS\tts\models\glow_tts.py:415: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=False):  # avoid mixed_precision in criterion

   --> TIME: 2025-05-05 23:19:07 -- STEP: 11/293 -- GLOBAL_STEP: 23925
     | > loss: 0.2884376049041748  (0.2957019860094244)
     | > log_mle: -0.3378767967224121  (-0.3274857130917636)
     | > loss_dur: 0.6263144016265869  (0.623187699101188)
     | > amp_scaler: 65536.0  (65536.0)
     | > grad_norm: tensor(2.4241, device='cuda:0')  (tensor(4.8814, device='cuda:0'))
     | > current_lr: 2.5e-07 
     | > step_time: 0.556  (0.5659539916298606)
     | > loader_time: 0.0021  (0.013859705491499468)


   --> TIME: 2025-05-05 23:19:18 -- STEP: 36/293 -- GLOBAL_STEP: 23950
     | > loss: 0.31276482343673706  (0.2747144483857685)
     | > log_mle: -0.307997465133667  (-0.3295642236868541)
     | > loss_dur: 0

...


   --> TIME: 2025-05-06 09:13:57 -- STEP: 97/293 -- GLOBAL_STEP: 82025
     | > loss: -0.08912146091461182  (-0.08984973135682722)
     | > log_mle: -0.47007322311401367  (-0.4832749366760254)
     | > loss_dur: 0.38095176219940186  (0.3934252053191981)
     | > amp_scaler: 2048.0  (2048.0)
     | > grad_norm: tensor(23.7368, device='cuda:0')  (tensor(18.6816, device='cuda:0'))
     | > current_lr: 6.975e-05 
     | > step_time: 0.3624  (0.3395075060657619)
     | > loader_time: 0.003  (0.022978310732497384)


   --> TIME: 2025-05-06 09:14:06 -- STEP: 122/293 -- GLOBAL_STEP: 82050
     | > loss: -0.10009652376174927  (-0.09019835439861795)
     | > log_mle: -0.49083244800567627  (-0.4797041875417115)
     | > loss_dur: 0.390735924243927  (0.38950583314309356)
     | > amp_scaler: 2048.0  (2048.0)
     | > grad_norm: tensor(14.4038, device='cuda:0')  (tensor(18.2490, device='cuda:0'))
     | > current_lr: 6.975e-05 
     | > step_time: 0.3555  (0.34356765278050155)
     | > loader_time

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.5181625366210938 (+0.018174934387207076)
     | > avg_loss: 0.044337505102157594 (+0.009016120433807374)
     | > avg_log_mle: -0.47297236919403074 (-0.002731013298034657)
     | > avg_loss_dur: 0.5173098742961884 (+0.011747133731841997)


 > EPOCH: 199/200
 --> TTS_training/run-May-05-2025_06+15PM-0000000

 > TRAINING (2025-05-06 09:15:52) 

   --> TIME: 2025-05-06 09:16:10 -- STEP: 4/293 -- GLOBAL_STEP: 82225
     | > loss: -0.1419474184513092  (-0.045491985976696014)
     | > log_mle: -0.5384880304336548  (-0.48441922664642334)
     | > loss_dur: 0.3965406119823456  (0.4389272406697273)
     | > amp_scaler: 2048.0  (2048.0)
     | > grad_norm: tensor(31.3166, device='cuda:0')  (tensor(21.1962, device='cuda:0'))
     | > current_lr: 7.000000000000001e-05 
     | > step_time: 0.3238  (0.3135295510292053)
     | > loader_time: 0.002  (0.47457098960876465)


   --> TIME: 2025-05-06 09:16:18 -- STEP: 29/293 -- GLOBAL_STEP: 82250
     | 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.5161861419677735 (-0.0019763946533203347)
     | > avg_loss: 0.048898065090179445 (+0.004560559988021851)
     | > avg_log_mle: -0.4696481943130493 (+0.003324174880981412)
     | > avg_loss_dur: 0.5185462594032287 (+0.001236385107040383)



Test the model!

In [13]:
import glob

output_path = "TTS_training"  # Change this to your actual output path
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])

# Get the latest checkpoint and config file
test_ckpt = ckpts[-1]  # Latest checkpoint
test_config = configs[-1]  # Corresponding config file

In [24]:
!tts --text "shel sem v trgovino in si kupil novo obleko" --model_path $test_ckpt --config_path $test_config --out_path out.wav

 > Using model: glow_tts
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024
 > Text: shel sem v trgovino in si kupil novo obleko
 > Text splitted to sentences.
['shel sem v trgovino in si kupil novo obleko']
 > Processing time: 0.9917700290679932
 > Real-time factor: 0.3094369642991461
 > Saving output to ou

In [25]:
import IPython
IPython.display.Audio("out.wav")